Test NB

In [10]:
import asyncio
import json
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from agents.prompts.unstructured_text_parser_message import UNSTRUCTURED_TEXT_PARSER_SYSTEM_MESSAGE
from autogen_ext.models.openai import OpenAIChatCompletionClient
from typing import List, Dict
import pandas as pd
import json
import pypdf
from dotenv import load_dotenv
import os


In [2]:
load_dotenv()

True

In [3]:

def extract_text_from_pdf(file_path: str) -> str:
    """
    Extracts all text content from a given PDF file.

    This tool's purpose is to provide the raw text data from a PDF to an LLM agent.
    The agent will then be responsible for writing code to parse this unstructured text.

    Args:
        file_path (str): The local path to the PDF file.

    Returns:
        str: A single string containing all the extracted text from the PDF.
             Returns an error message string if extraction fails.
    """
    try:
        reader = pypdf.PdfReader(file_path)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text() + "\n--- End of Page ---\n"
        return full_text
    except Exception as e:
        return f"Error extracting text from PDF: {str(e)}"


In [4]:
parsed_text = extract_text_from_pdf('temp/test_statement.pdf')

In [5]:
parsed_text

" \n \n863300\nMOHIT AGGARWAL\n \n15237 BROOKSTONE ST\n \nFONTANA  CA  92336-4051\nCostco Anywhere Visa\n®\n Card by Citi\nMOHIT AGGARWAL\nMember Since 2014  Account number ending in: 8633 \nBilling Period: \n 07/02/25-08/01/25\nBilling Inquiries and Customer Service\nPO Box 790046 ST. LOUIS, MO 63179-0046\n1-855-378-6467, (TTY: 711)\nwww.citicards.com\nAUGUST  STATEMENT\nMinimum payment due:\n \n$41.00\nNew balance as of 08/01/25:\n \n$910.51\nPayment due date:\n \n08/27/25\nLate Payment Warning:\nIf we do not receive your Minimum Payment by the\n \ndate listed above, you may have to pay a late fee of up to $41 and your APRs\n \nmay be increased up to the Penalty APR of 29.99%.\nFor information about credit counseling services, call 1-877-337-8187 (TTY: 711). \nAccount Summary\nPrevious balance\n \n$1,041.44\nPayments\n-\n$1,041.44\nCredits\n-\n$36.28\nPurchases\n \n+$946.79\nCash advances\n \n+$0.00\nFees\n \n+$0.00\nInterest\n \n+$0.00\nNew balance\n \n$910.51\nCredit Limit\nCredit 

" \n \n863300\nMOHIT AGGARWAL\n \n15237 BROOKSTONE ST\n \nFONTANA  CA  92336-4051\nCostco Anywhere Visa\n®\n Card by Citi\nMOHIT AGGARWAL\nMember Since 2014  Account number ending in: 8633 \nBilling Period: \n 07/02/25-08/01/25\nBilling Inquiries and Customer Service\nPO Box 790046 ST. LOUIS, MO 63179-0046\n1-855-378-6467, (TTY: 711)\nwww.citicards.com\nAUGUST  STATEMENT\nMinimum payment due:\n \n$41.00\nNew balance as of 08/01/25:\n \n$910.51\nPayment due date:\n \n08/27/25\nLate Payment Warning:\nIf we do not receive your Minimum Payment by the\n \ndate listed above, you may have to pay a late fee of up to $41 and your APRs\n \nmay be increased up to the Penalty APR of 29.99%.\nFor information about credit counseling services, call 1-877-337-8187 (TTY: 711). \nAccount Summary\nPrevious balance\n \n$1,041.44\nPayments\n-\n$1,041.44\nCredits\n-\n$36.28\nPurchases\n \n+$946.79\nCash advances\n \n+$0.00\nFees\n \n+$0.00\nInterest\n \n+$0.00\nNew balance\n \n$910.51\nCredit Limit\nCredit Limit\n \n$7,000\nIncludes $1,400.00 cash advance limit\nAvailable Credit Limit\n \n$6,089\nIncludes $1,400\n \navailable for cash advance\nCostco Cash Back\n \nRewards Summary\nas of 08/01/25\n$179.26\n» See page 3 for more information\n \nabout your rewards\nFor Payments, send check to: Citi Cards, PO BOX 60734, City of Industry CA, 91716-0734\nCostco Anywhere Visa\n®\n Card\nPO Box 790057\n \nSaint Louis, MO 63179-0057\nYour Monthly Statement\n \nis Enclosed\nPay your bill from virtually anywhere with the\n \nCiti Mobile\n®\n App and Citi\n®\n Online\nTo download:\n \nText \n'App15'\n to \nMyCiti (692484)\n \nor go to your device's app store.\n \nOr visit \nwww.citicards.com\n \n   \nMinimum payment due\n \n$41.00\nNew balance\n \n$910.51\nPayment due date\n \n08/27/25\nAmount enclosed: \n$\nAccount number ending in 8633 \nPlease make check payable to Citi Cards.\nCiti Cards\n \nPO BOX 60734\n \nCity of Industry CA 91716-0734\n  \n--- End of Page ---\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n           \n           \n863300\nCARDHOLDER SUMMARY\nMOHIT AGGARWAL\n \nCard ending in 8633\nNew Charges\n \n$451.75\nHIMANI SOOD\n \nCard ending in 1325\nNew Charges\n \n$495.04\nACCOUNT SUMMARY\nSale\n \nDate\nPost\n \nDate\nDescription\nAmount\nPayments, Credits and Adjustments\n07/22\nELECTRONIC PAYMENT-THANK YOU\n-\n$1,041.44\n06/30\n07/02\nHEADWAY                  NEW YORK     NY\n-\n$10.00\n07/10\n07/10\nHEADWAY                  NEW YORK     NY\n-\n$10.00\n07/16\n07/16\nCOSTCO WHSE #0627        FONTANA      CA\n-\n$16.28\nMOHIT AGGARWAL \nStandard Purchases\n07/05\n07/05\nHEADWAY                  HEADWAY.CO   NY\n $10.00 \n07/06\n07/06\nCOSTCO WHSE #0678        RANCHO CUCAMOCA\n $312.41 \n07/12\n07/12\nTRADER JOE S #217        RANCHO CUCAMOCA\n $66.64 \n07/12\n07/12\nSQ *HANDEL'S RANCHO CUCAMRancho\n \nCucamoCA\n $13.50 \n07/13\n07/13\nHEADWAY                  HEADWAY.CO   NY\n $10.00 \n07/13\n07/13\nDESI BAZAAR CASH AND CAR RANCHO\n \nCUCAMOCA\n $1.99 \n07/13\n07/13\nHOMEGOODS 275            RCH CUCAMONGACA\n $18.31 \n07/14\n07/14\nALDI 79061               FONTANA      CA\n $8.91 \n07/14\n07/14\nRING STANDARD PLAN       RING.COM     CA\n $9.99 \nHIMANI SOOD \nStandard Purchases\n07/03\n07/03\nCOSTCO GAS #0686         MONTCLAIR    CA\n $42.49 \n07/08\n07/08\nCOSTCO WHSE #0627        FONTANA      CA\n $2.71 \n07/08\n07/08\nCOSTCO WHSE #0627        FONTANA      CA\n $145.98 \n07/14\n07/14\nCOSTCO GAS #0686         MONTCLAIR    CA\n $40.09 \n07/16\n07/16\nCOSTCO WHSE #0627        FONTANA      CA\n $186.02 \n07/17\n07/17\nUBER   *EATS             HELP.UBER.COMCA\n $35.31 \n07/24\n07/24\nCOSTCO GAS #0478         SAN BERNARDINCA\n $42.44 \nFees Charged\nTOTAL FEES FOR THIS PERIOD\n$0.00\nInterest Charged\nTOTAL INTEREST FOR THIS PERIOD\n$0.00\n2025 totals year-to-date\nTotal fees charged in 2025\n \n$0.00\nTotal interest charged in 2025\n \n$0.00\nCostco Cash Back\n \nRewards Summary\nTotal Costco\n \nCash Back Rewards Balance:\n$179.26\nCostco Cash Back Rewards Summary\nCostco Cash Back Rewards balance as of\n \nlast statement   .................................... +$158.15\nEarned this period .................................. +$21.11\nTotal Costco Cash Back Rewards Balance\n \nYear to Date :\n \n$179.26\nCostco Cash Back Rewards\n \nEarned This Period\n5% on gas at Costco\n............................. +$6.24\n4% on other eligible gas\n \nand EV charging....................................... +$0.00\n5% and 4% earn is on a combined $7,000\n \nspend per year, 1% thereafter\n3% on restaurants.................................... +$1.06\n3% on eligible travel ............................... +$0.00\n2% on Costco and Costco.com\n \npurchases ................................................. +$12.61\n1% on all other purchases....................... +$1.20\nTotal Earned:\n \n$21.11\n» Visit \nCiti.com/Costco\n \nfor more information\nPage 3 of 4\nwww.citicards.com\nCustomer Service\n \n1-855-378-6467\n(TTY: 711)\nMOHIT AGGARWAL\n--- End of Page ---\n"

In [6]:
type(parsed_text)

str

In [7]:
# --- 1. Imports ---
# Standard library and Pydantic
import os
import json
from typing import List, Optional
from pydantic import BaseModel, Field

# Core AutoGen components with explicit paths
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_core.tools import Workbench

# --- 2. Define the Desired JSON Structure ---
# This defines the schema for each individual transaction.
class Transaction(BaseModel):
    cardholder: str = Field(description="The name of the cardholder who made the transaction.")
    sale_date: Optional[str] = Field(description="The date the transaction was made, e.g., '07/15'")
    post_date: str = Field(description="The date the transaction was posted to the account, e.g., '07/16'")
    description: str = Field(description="The description of the transaction")
    transaction_amount: float = Field(description="The amount. Debits (purchases) must be negative, credits (payments) must be positive.")

# This is the main structure for the entire statement.
class StatementData(BaseModel):
    bank_name: str = Field(description="The name of the bank or card issuer, e.g., 'Costco Anywhere Visa Card by Citi'")
    transactions: List[Transaction] = Field(description="A list of all transactions from the statement")

# --- 3. Create the Python Function (The 'Tool') ---
def parse_bank_statement(bank_name: str, transactions: List[dict]) -> str:
    """
    Parses raw bank statement data into a structured JSON format using Pydantic models.
    
    Args:
        bank_name: The name of the bank or card issuer.
        transactions: A list of dictionaries, each representing a transaction.
                      Each dictionary must contain all fields for the Transaction model.
                      
    Returns:
        A JSON string representation of the structured data.
    """
    statement = StatementData(
        bank_name=bank_name,
        transactions=[Transaction(**t) for t in transactions]
    )
    return statement.model_dump_json(indent=2)

# --- 4. Configure AutoGen Agents and Environment ---
# Ensure your OPENAI_API_KEY is set as an environment variable
config_list = [
    {
        "model": "gpt-4o", 
        "api_key": os.getenv("OPENAI_API_KEY2")
    }
]

# The Workbench manages the tools and execution environment.
workbench = Workbench(work_dir="coding", tools=[parse_bank_statement])

# The UserProxyAgent acts as the executor.
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    workbench=workbench,
)

# The AssistantAgent acts as the planner, deciding to use the tool.
parser_assistant = AssistantAgent(
    name="ParserAssistant",
    system_message="""You are an expert financial data extractor. Your task is to meticulously analyze the user's text and use the `parse_bank_statement` tool to convert it into a structured JSON format.

    CRITICAL INSTRUCTIONS:
    1.  **Cardholders:** The statement has multiple cardholders (e.g., MOHIT AGGARWAL, HIMANI SOOD). You MUST associate each transaction with the correct cardholder name it's listed under.
    2.  **Transaction Amounts:** You MUST correctly interpret the signs. 
        -   Purchases are DEBITS and must be represented as **NEGATIVE** numbers.
        -   Payments and Credits are CREDITS and must be represented as **POSITIVE** numbers. If a payment is shown as '-$1,041.44', its value is 1041.44.
    3.  **Data Focus:** Ignore all summary data, ads, addresses, and page markers. Focus ONLY on the detailed list of transactions.""",
    llm_config={"config_list": config_list},
    tools=[parse_bank_statement],
)

# --- 5. Your Input String is Loaded Here ---
pdf_text_string = parsed_text

# --- 6. Initiate the Conversation and Print the Final Output ---
user_proxy.initiate_chat(
    recipient=parser_assistant,
    message=f"Please parse the following bank statement text:\n\n{pdf_text_string}"
)

final_message = user_proxy.last_message(parser_assistant)
final_json_output = final_message.get("content", "")

print("\n--- Final Parsed JSON Output ---")
print(final_json_output)

TypeError: Workbench() takes no arguments

In [13]:
# Install (run once)

# ---------------------------
# Agent + Code-executor demo
# ---------------------------
import os, asyncio
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import TextMessage

async def run_parse_v0_7_2(statement_text: str):
    # 1) model client used by AssistantAgent
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o", 
        api_key=os.getenv("OPENAI_API_KEY2")
    )

    # 2) assistant - this agent will generate parsing code
    assistant = AssistantAgent(
        name="assistant",
        model_client=model_client,
        system_message=(
            "You are an expert Python developer. Given a raw multi-line bank statement string, "
            "produce Python code inside triple-backtick ```python``` blocks that parses the statement "
            "into a JSON object with keys: account_info, transactions, rewards and also prints a pandas DataFrame "
            "for transactions. When done, print the JSON and then reply with the single word: TERMINATE"
        ),
        reflect_on_tool_use=True,
    )

    # 3) code executor component (local) — start it before using
    code_executor = LocalCommandLineCodeExecutor(work_dir="coding")
    await code_executor.start()

    # 4) code executor *agent* (do NOT pass llm_config)
    code_executor_agent = CodeExecutorAgent(
        name="code_executor",
        code_executor=code_executor,
        # optional: model_client=..., approval_func=..., supported_languages=...
        approval_func=None,
    )

    # 5) team: assistant -> code executor (round-robin). Terminate on 'TERMINATE' or after N messages
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(6)
    team = RoundRobinGroupChat(participants=[assistant, code_executor_agent], termination_condition=termination)

    # 6) the task: feed the raw statement and ask for JSON + DataFrame
    task_msg = TextMessage(
        content=(
            "Parse the bank statement below into JSON & a pandas DataFrame. "
            "Print the JSON first, then print the DataFrame. End your final response with TERMINATE.\n\n"
            f"{statement_text}"
        ),
        source="user",
    )

    # 7) run the team (synchronous top-level via asyncio)
    result = await team.run(task=task_msg)
    # final model text available as result.chat_message
    print("===== AGENT FINAL OUTPUT =====")
    print(result.chat_message.content)

    # cleanup
    await code_executor.stop()



In [14]:
# Usage: replace the string below with your extracted-statement string
YOUR_STATEMENT = parsed_text
asyncio.run(run_parse_v0_7_2(YOUR_STATEMENT))

RuntimeError: asyncio.run() cannot be called from a running event loop